# String Searching Algorithms

String matching algorithms are algorithms that find all occurrences of a given pattern string in a text string. These algorithms have a wide range of applications, such as in text editing, data compression, and search engines.

There are several different string matching algorithms, each with its own advantages and disadvantages. Some of the most commonly used algorithms include:

1. Naive String Matching: This algorithm checks every substring of the text against the pattern, which is the simplest approach but also the least efficient.

2. Knuth-Morris-Pratt (KMP) Algorithm: This algorithm uses the prefix function to avoid unnecessary comparisons of characters in the text string and the pattern.

3. Boyer-Moore Algorithm: This algorithm uses two preprocessing steps to speed up the matching process: a bad character rule and a good suffix rule.

4. Rabin-Karp Algorithm: This algorithm uses hashing to check if the pattern matches a substring of the text string.

5. Finite Automaton Algorithm: This algorithm uses a finite state machine to recognize the pattern in the text string.

These algorithms have different time and space complexities, and the choice of which algorithm to use depends on the specific requirements of the application.

## Practical Applications

* Search engines: Search engines use string matching algorithms to find relevant web pages based on the user's query.

* Virus scanners: Virus scanners use string matching algorithms to detect malicious code in files by searching for known virus signatures.

* Data compression: Compression algorithms use string matching algorithms to identify repeated patterns in the data, which can then be replaced with a shorter code.

* Text editors: Text editors use string matching algorithms to implement search and replace functionality.

* Natural language processing: String matching algorithms are used in natural language processing tasks such as named entity recognition, where a specific pattern of words or characters is matched to identify named entities such as people, organizations, and locations.

* DNA sequence analysis: String matching algorithms are used in bioinformatics to search for specific patterns in DNA sequences.

## Naive String Matching

The naive string matching algorithm is the simplest string matching algorithm. It checks every substring of the text string against the pattern string. If the pattern string is found, the index of the first character of the substring is added to the list of matches. If the pattern string is not found, the algorithm continues to the next substring.


## Naive String Matching Algorithm

The naive_string_matcher function takes two input strings text and pattern, and returns a list of all occurrences of the pattern in the text.

The function first initializes the lengths of the text and pattern, and creates an empty list occurrences to store the indices of each occurrence of pattern in text.

It then iterates through every possible starting index of a substring of text that is the same length as pattern. For each starting index i, it checks whether the substring of text starting at index i and with length m (i.e., the same length as pattern) is equal to pattern. If it is, then it appends i to the occurrences list.

Finally, the function returns the occurrences list containing the indices of each occurrence of pattern in text.

In [1]:
text = "Riga Rocks RBS Rocking as well also RBS is great plus we have RBS at home"
pattern = "RBS"
# Python built in
text.find(pattern), text.rfind(pattern) # of course if you have more than two occurences you would need to write some more code

(11, 62)

In [8]:
# let's return indexes of all occurences of pattern in text
def find_all_naive(text, pattern):
    indexes = []
    i = 0
    while i < len(text):
        i = text.find(pattern, i) # find returns -1 if pattern is not found
        if i == -1:
            break
        indexes.append(i)
        i += 1
    return indexes

indexes = find_all_naive(text, pattern)
indexes

[11, 36, 62]

In [9]:
text.find("willnotexist")

-1

In [9]:
def find_all_indexes_using_built_in_find(text, pattern):
    occurences = []
    find_index = text.find(pattern)
    while find_index >=0:
        occurences.append(find_index)
        find_index = text.find(pattern, find_index+1) # so we keep looking at slices
    return occurences

In [13]:
find_all_indexes_using_built_in_find(text,pattern)

[11, 36, 62]

In [10]:
## Implementation of naive string matching

def naive_string_matcher(text, pattern):
    n = len(text)
    m = len(pattern)
    occurrences = [] # storage of found patterns

    for i in range(n - m + 1):
        if text[i:i + m] == pattern: # this equality actually might be efficient..
        # to make it completely naive we would need to compare each character one by one
            occurrences.append(i)

    return occurrences

# so this has O(n*m) complexity


In [11]:
naive_string_matcher(text, pattern)

[11, 36, 62]

## Knuth-Morris-Pratt (KMP) Algorithm

Idea behind KMP algorithm is to not match a character more than once. If we have matched a character in the pattern, we can skip some characters in the text string and start matching from the next character in the pattern.

We will perform pre-processing on the pattern string to create a prefix function that will help us determine the number of characters to skip in the text string when a mismatch occurs.

This prefix function will essentially be a DFA (Deterministic Finite Automaton) that will help us determine the next state to go to when a mismatch occurs.

In [14]:
# let's implement KMP algorithm

def compute_prefix_function(pattern):
    m = len(pattern)
    pi = [0] * m
    k = 0
    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k
    return pi

def kmp_matcher(text, pattern, pi=None):
    n = len(text)
    m = len(pattern)
    if pi is None:
        pi = compute_prefix_function(pattern)
    q = 0
    occurrences = []
    for i in range(n): # so you see this is complexity O(n)
        while q > 0 and pattern[q] != text[i]:
            q = pi[q - 1]
        if pattern[q] == text[i]:
            q += 1
        if q == m:
            occurrences.append(i - m + 1)
            q = pi[q - 1]
    return occurrences

# test it
kmp_matcher(text, pattern)

[11, 36, 62]

In [6]:
#let's make a bit text 1000 time bigger
text1k = text * 1000
# our pattern will be our text
pattern1k = text
# so length of n
print(f"length of text(n): {len(text1k)}, pattern(m): {len(pattern1k)}")

length of text(n): 73000, pattern(m): 73


In [18]:
%%timeit
naive_string_matcher(text1k, pattern1k)

13.1 ms ± 551 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
kmp_matcher(text1k, pattern1k)

21.5 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Preliminary Results

So our Naive match actually beats KMP? Why? Because each time we recomputed the prefix function. We can actually precompute the prefix function and then use it to match the pattern with the text.

In [15]:
# now let's precompute prefix function
pi = compute_prefix_function(pattern1k)

In [19]:
%%timeit
kmp_matcher(text1k, pattern1k, pi)

17.4 ms ± 852 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
results = kmp_matcher(text1k, pattern1k, pi)
len(results)

1000

In [21]:
results_naive = naive_string_matcher(text1k, pattern1k)
len(results_naive)

1000

In [23]:
# first 5 results
results[:5], results_naive[:5]

([0, 73, 146, 219, 292], [0, 73, 146, 219, 292])

In [24]:
# last 5 results
results[-5:], results_naive[-5:]

([72635, 72708, 72781, 72854, 72927], [72635, 72708, 72781, 72854, 72927])

### More conclusions

Our naive match is still  faster than KMP. Why? 
One hypothesis is that == is more efficient than comparing strings one character at a time.

### Needle matters!

Our needle "Riga Rocks RBS Rocking as well also RBS is great plus we have RBS at home" will draw a mismatch very quickly even with naive match because we only need to check R or Ri at most , so no more than 2 characters.

In [25]:
# So our pattern will be Riga * 100 + "X"
patternX = "Riga" * 100 + "X"
# our text will be Riga * 1000+ "X" * 100
textX = ("Riga" * 1000 + "X") * 100
# lengths
print(f"length of text(n): {len(textX)}, pattern(m): {len(patternX)}")

length of text(n): 400100, pattern(m): 401


In [26]:
# test results
results = kmp_matcher(textX, patternX)
len(results)

100

In [27]:
# naive
results_naive = naive_string_matcher(textX, patternX)
len(results_naive)

100

In [28]:
# lets precoompute prefix function
pi = compute_prefix_function(patternX)


In [29]:
results = kmp_matcher(textX, patternX, pi)
len(results)

100

In [30]:
%%timeit
naive_string_matcher(textX, patternX)

85.3 ms ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%timeit
kmp_matcher(textX, patternX, pi)

108 ms ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Random text creation

In [32]:
# let's create a function that retuns some random text from some alphabet
import random
import string # for ascii
alpha = string.ascii_lowercase+string.ascii_uppercase+string.digits+string.punctuation+" "
def gen_random_text(n, alphabet=alpha):
    return ''.join(random.choice(alphabet) for _ in range(n))

# test with 100
text = gen_random_text(100)
text

'NXz{gi_Dp1>8YS}b+9W<`s,n]6s!L6V^ocKhU;dVS.iYz45FFM\\4^1(jLmL~Co$B8`\\1XSu_&\\CJx"WG.M7JME%+:I2+3N#PEY;E'

### Library of Babel

https://en.wikipedia.org/wiki/The_Library_of_Babel

![Babel](https://upload.wikimedia.org/wikipedia/en/a/ae/The_library_of_babel_-_bookcover.jpg)

<iframe width="560" height="315" src="https://www.youtube.com/embed/no_elVGGgW8?si=5tmekCUwWoo-JPwu" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [33]:
# so lets gen 10M text
text10M = gen_random_text(10**7)
pattern100 = gen_random_text(10**2)

# lengths
print(f"length of text(n): {len(text10M)}, pattern(m): {len(pattern100)}")

length of text(n): 10000000, pattern(m): 100


In [34]:
# now let's test it
results = kmp_matcher(text10M, pattern100)
len(results)

0

In [35]:
naive_results = naive_string_matcher(text10M, pattern100)
len(naive_results)

0

In [36]:
# we are allowed to precompute
pi = compute_prefix_function(pattern100)
# so jumps are all 0 meaning we do not gain anythin from any reoccurence
pi

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [37]:
%%timeit
naive_string_matcher(text10M, pattern100)

1.91 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
kmp_matcher(text10M, pattern100, pi)

1.69 s ± 48.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Boyer-Moore Algorithm

The Boyer-Moore algorithm is a string matching algorithm that uses two preprocessing steps to speed up the matching process: a bad character rule and a good suffix rule.

The bad character rule works by examining the character in text that caused the mismatch with pattern, and then shifting pattern so that the character in pattern that matches the bad character in text is aligned with the bad character in text. This can be done because we know that any occurrences of pattern in text that end at the index of the bad character in text cannot match due to the mismatch.

The good suffix rule works by examining the longest suffix of pattern that matches a suffix of the current mismatch. If such a suffix exists, we can shift pattern so that the matching suffix is aligned with the mismatched suffix in text. This can be done because we know that any occurrences of pattern in text that end at the index of the mismatched suffix cannot match due to the mismatch.

The Boyer-Moore algorithm combines these two rules to determine the best shift to make after a mismatch occurs. Specifically, it chooses the larger of the shifts suggested by the bad character rule and the good suffix rule. This means that the algorithm skips over as many characters as possible in text before attempting another match.

In practice, the Boyer-Moore algorithm is often faster than other string matching algorithms, particularly when the pattern string is long or there are many occurrences of the pattern in the text. However, it can be slower than other algorithms in certain cases, such as when the pattern string is short or there are few occurrences of the pattern in the text.

In [42]:
def boyer_moore(text, pattern, debug=False):
    n = len(text)
    m = len(pattern)
    if m == 0:
        return 0

    # Initialize variables
    skip = [m] * 256
    for i in range(m - 1):
        skip[ord(pattern[i])] = m - i - 1

    # Search for the pattern in the text
    i = m - 1
    while i < n:
        j = m - 1
        while text[i] == pattern[j]:
            if debug:
                print(f"Potential match at location {i} letter {text[i]}")
            if j == 0:
                return i # we could modif this to collect indexes
            i -= 1
            j -= 1
        i += max(skip[ord(text[i])], m - j)
        if debug:
            print(f"skipping to position {i}")

    return -1

In [51]:
# we will modify Boyer-Moore to return all indexes
def boyer_moore_all(text, pattern, debug=False):
    n = len(text)
    m = len(pattern)
    if m == 0:
        return []

    # Initialize variables
    skip = [m] * 256
    for i in range(m - 1):
        skip[ord(pattern[i])] = m - i - 1

    # Search for the pattern in the text
    i = m - 1
    indexes = []
    while i < n:
        j = m - 1
        while text[i] == pattern[j]:
            if debug:
                print(f"Potential match at location {i} letter {text[i]}")
            if j == 0:
                indexes.append(i)
                break
            i -= 1
            j -= 1
        i += max(skip[ord(text[i])], m - j)
        if debug:
            print(f"skipping to position {i}")

    return indexes

# test it on Riga Riga Riga Rocks Riga
text = "Good old Riga Riga Riga Rocks Riga"
pattern = "Riga"
boyer_moore_all(text, pattern)

[9, 14, 19, 30]

In [44]:
# print(text, pattern)
# print lenth
print(f"length of text(n): {len(text)}, pattern(m): {len(pattern)}")
boyer_moore(text, pattern)

length of text(n): 100, pattern(m): 3


-1

In [45]:
# let's do 10mtext 
boyer_moore(text10M, pattern100)

-1

In [47]:
%%timeit
boyer_moore(text10M, pattern100)

80.2 ms ± 4.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Comments on Boyer-Moore results

We can see that Boyer Moore beats both Naive and KMP. This is because Boyer Moore is a more efficient algorithm than Naive and KMP. It uses two preprocessing steps to speed up the matching process: a bad character rule and a good suffix rule. This allows it to skip over as many characters as possible in the text before attempting another match, making it faster than Naive and KMP in many cases.

Of course, this was not completely fair comparison beause naive and KMP here had an option to collect matches, while B-M did not.

However, in for these particular 10M characters there was no match for the pattern so the results are still valid.

In [52]:
%%timeit
boyer_moore_all(text10M, pattern100)

95.5 ms ± 6.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Final Conclusions on Boyer-Moore

We see that even after adding the match collection to Boyer-Moore, it is still faster than Naive and KMP.

In [48]:
boyer_moore("abra dabra cadabra", "cada")

11

In [23]:
boyer_moore("a quick brown fox jumped over a sleeping dog and ate it", "sleeping dog")

skipping to position 23
skipping to position 25
skipping to position 26
skipping to position 38
skipping to position 43
Potential match at location 43 letter g
Potential match at location 42 letter o
Potential match at location 41 letter d
Potential match at location 40 letter  
Potential match at location 39 letter g
Potential match at location 38 letter n
Potential match at location 37 letter i
Potential match at location 36 letter p
Potential match at location 35 letter e
Potential match at location 34 letter e
Potential match at location 33 letter l
Potential match at location 32 letter s


32

## Boyer-Moore efficiency

So Boyer Moore is quite efficient Omega(n/m) in real life applications where pattern is large.

### Boyer-Moore Algorithm explanation

The boyer_moore function takes two input strings text and pattern, and returns the index of the first occurrence of pattern in text. If pattern does not occur in text, the function returns -1.

The function first initializes the lengths of the text and pattern, and checks if the length of pattern is zero. If it is, it returns 0 (indicating that pattern occurs at the beginning of text).

It then initializes a skip table skip that stores the number of characters to skip when a mismatch occurs, based on the character that caused the mismatch. The skip table is initialized with the value m for each character in the ASCII table.

Next, the function iterates through the first m - 1 characters of pattern and updates the skip table with the appropriate values for each character in pattern.

Finally, the function searches for pattern in text. It starts at the end of pattern and the corresponding index in text, and works its way backwards through pattern and text until a mismatch is found. If the mismatch occurs at the first character of pattern, then the function returns the current index in text. Otherwise, the function jumps ahead in text by the maximum of the skip value for the mismatched character and the number of characters remaining in pattern.

If pattern is not found in text, the function returns -1.

## Other String Matching Algorithms

There are several other string matching algorithms, including the Knuth-Morris-Pratt (KMP) algorithm, the Rabin-Karp algorithm, and the finite automaton algorithm. These algorithms have different time and space complexities, and the choice of which algorithm to use depends on the specific requirements of the application.

## TODO What algorithm is Python Find/Index using